In [27]:
from pybit.unified_trading import HTTP

# اتصال به Bybit (در حالت تست‌نت)
session = HTTP(testnet=True)

# گرفتن اطلاعات آخرین قیمت
response = session.get_index_price_kline(
    category="linear",   # نوع قرارداد (linear برای فیوچرز)
    symbol="BTCUSDT",    # نماد بازار (BTC/USDT)
    interval=1,         # تایم‌فریم (60 دقیقه)
    limit=1               # تعداد داده‌ها، فقط یک کندل آخر
)

# نمایش پاسخ
print(response)

{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1740490440000', '89023.75', '89024.67', '88989.7', '88989.81']]}, 'retExtInfo': {}, 'time': 1740490453292}


In [16]:
from pybit.unified_trading import HTTP
session = HTTP(testnet=True)
print(session.get_mark_price_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=60,
    limit =1
))

{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSDT', 'category': 'linear', 'list': [['1740484800000', '39217.76', '79878.2', '39192.12', '77399.2']]}, 'retExtInfo': {}, 'time': 1740487453453}
